<a href="https://colab.research.google.com/github/kobars/Assignment-05-W05-First-Machine-Learning-Project/blob/notebook/Assignment_05_W05_First_Machine_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this colabs, we try to do a binary classification to determine whether a video published on YouTube will reach a certain number of views. The dataset is taken from Kaggle which can be seen from the following link https://www.kaggle.com/datasnaek/youtube-new

# Use the right version of TensorFlow

The following hidden code cell ensures that the Colab will run on TensorFlow 2.X.

In [0]:
#@title Run on TensorFlow 2.x
%tensorflow_version 2.x

# Call the import statements

The following code imports the necessary modules.

In [61]:
#@title Load the imports

# from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import tensorflow as tf
import string,math,pickle
from textblob import TextBlob
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

Ran the import statements.


In [0]:
df1 = pd.read_csv('https://raw.githubusercontent.com/kobars/Assignment-05-W05-First-Machine-Learning-Project/master/ID_videos.csv')

In [55]:
df1

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,o4MCAySZ0Hg,"LET'S MAKE IT HAPPEN, 2020",2020-03-27T06:02:30.000Z,UCepgnl-TtJ8DurHdC6EE22w,Ria SW,24,20.28.03,[none],1110686,126750,1149,10526,https://i.ytimg.com/vi/o4MCAySZ0Hg/default.jpg,False,False,Tahun 2019 adalah tahun paling berat buatku. A...
1,sVWyQUZOmLc,GOOGLE KALAH TEKA TEKI||BOCAH KORSLET,2020-03-23T13:47:16.000Z,UCMJozRNsVN9KkaUXokUP9eQ,Ronsi Geronsiyono,24,20.28.03,Bocah Korslet|Bocah|Korslet|Bahasa Inggris|kom...,8442248,257400,13168,12000,https://i.ytimg.com/vi/sVWyQUZOmLc/default.jpg,False,False,Terima kasih sudah menontonJangan lupa subscri...
2,ZdjSIBjDspU,TEGAS! Pesan Persatuan dan Kritik Dokter Tirta...,2020-03-24T23:00:11.000Z,UCYEa4_MC7OFjcWrO6SO-u8g,Indonesia Lawyers Club,25,20.28.03,tvone|tv one|tvonenews|indonesia lawyers club|...,3627883,131862,2676,20810,https://i.ytimg.com/vi/ZdjSIBjDspU/default.jpg,False,False,"Jakarta, tvOnenews.com - Dokter Tirta menyampa..."
3,bBxWId6hfuw,Alasan Tegar Septian Menikah Diumur 18 Tahun,2020-03-25T06:30:02.000Z,UCo1CFjot3CX97sn_ngJnGPg,Ini Talk Show,23,20.28.03,net|media|netmedia|netmediatama|indonesia|jaka...,6365021,98615,4462,12162,https://i.ytimg.com/vi/bBxWId6hfuw/default.jpg,False,False,Ini Talk Show adalah program talk show masa ki...
4,DmJZNdlC8IQ,ORANG TINGGI VS ORANG PENDEK || Situasi Memalu...,2020-03-24T07:00:04.000Z,UCGXnDxirLuSrIHwQXWxRzeg,123 GO! Like Indonesian,24,20.28.03,123 go|123 GO!|123 GO! Like|123 GO! Like Indon...,1625720,14068,2174,911,https://i.ytimg.com/vi/DmJZNdlC8IQ/default.jpg,False,False,"Orang tinggi punya masalah, orang pendek juga ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,QgxU6xx1t0g,NYENTUH BANGET SOAL HATI :( - SEDIHAKUNYA*BAPER,2020-03-27T01:27:42.000Z,UCsQaQulkx6bkbWJC3wtIaTg,Maskray,24,20.28.03,#maskrayy1|#ikrayy|#maskray|#ikke.jenner,94415,8711,118,325,https://i.ytimg.com/vi/QgxU6xx1t0g/default.jpg,False,False,Follow my instagram : maskrayy1 with ikke.jenn...
146,OSRa0d7z9rU,RISA SOLIHAH Cover AISYAH ISTRI RASULULLAH,2020-03-24T09:53:01.000Z,UCRCIeN_AoML5twbpWpXR5FA,TMD Media Religi,10,20.28.03,aisyah|risasholihah|lagureligi|suaramerdu|wani...,244031,5603,103,321,https://i.ytimg.com/vi/OSRa0d7z9rU/default.jpg,False,False,Deskripsi Alhamdulillah akhirnya Risa Solihah ...
147,bsz5hgKTyvY,Yessiel Trivena - Demi Raga Yang Lain (Officia...,2020-03-19T15:33:19.000Z,UC2G22-5R0-KCQGKJOUPktYQ,Eka Gustiwana,10,20.28.03,eka gustiwana|lagu tenaga medis|lagu corona|la...,827910,68309,229,3643,https://i.ytimg.com/vi/bsz5hgKTyvY/default.jpg,False,False,Donasi via KitaBisa.com untuk mendukung para t...
148,scke3FBGP0Y,5 HAPE PALING HOT DI INDONESIA SAAT INI 🔥,2020-03-27T06:29:23.000Z,UCtaSWbg8OHxg6h10axQXV-g,DroidLime,28,20.28.03,droidlime|review gadget|jujur|keras|bahasa|ind...,81566,2735,91,362,https://i.ytimg.com/vi/scke3FBGP0Y/default.jpg,False,False,"GIVEAWAY IPHONE 11!..Hi guys, udah cuci tangan..."


In [0]:
df2 = pd.read_csv('https://raw.githubusercontent.com/kobars/Assignment-05-W05-First-Machine-Learning-Project/master/ID_videos2.csv')

In [57]:
df2

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,aN0ZnoRg_IY,SABYAN - AISYAH | COVER,2020-03-28T02:10:45.000Z,UCeZA77FZSG4YzBcvBJ9eynA,SABYAN,10,20.29.03,Nissa sabyan|sabyan gambus|sabyan aisyah|ya al...,1833030,221557,2548,27195,https://i.ytimg.com/vi/aN0ZnoRg_IY/default.jpg,False,False,Assalamualaikum wr. wb.Alhamdulillah hari ini ...
1,o4MCAySZ0Hg,"LET'S MAKE IT HAPPEN, 2020",2020-03-27T06:02:30.000Z,UCepgnl-TtJ8DurHdC6EE22w,Ria SW,24,20.29.03,[none],1593497,148105,2241,12471,https://i.ytimg.com/vi/o4MCAySZ0Hg/default.jpg,False,False,Tahun 2019 adalah tahun paling berat buatku. A...
2,ZdjSIBjDspU,TEGAS! Pesan Persatuan dan Kritik Dokter Tirta...,2020-03-24T23:00:11.000Z,UCYEa4_MC7OFjcWrO6SO-u8g,Indonesia Lawyers Club,25,20.29.03,tvone|tv one|tvonenews|indonesia lawyers club|...,4028118,143536,2855,22160,https://i.ytimg.com/vi/ZdjSIBjDspU/default.jpg,False,False,"Jakarta, tvOnenews.com - Dokter Tirta menyampa..."
3,lTnjVMV9jmY,KENAPA CUMA LOE YG BERANI NGOMONG?! JAKARTA SU...,2020-03-28T03:57:47.000Z,UCYk4LJI0Pr6RBDWowMm-KUw,Deddy Corbuzier,24,20.29.03,anies baswedan|jokowi|Covid19|Indonesia|jakart...,3211468,171080,7155,44498,https://i.ytimg.com/vi/lTnjVMV9jmY/default.jpg,False,False,#indonesia #corona #covid19
4,bBxWId6hfuw,Alasan Tegar Septian Menikah Diumur 18 Tahun,2020-03-25T06:30:02.000Z,UCo1CFjot3CX97sn_ngJnGPg,Ini Talk Show,23,20.29.03,net|media|netmedia|netmediatama|indonesia|jaka...,7218362,108221,5083,13054,https://i.ytimg.com/vi/bBxWId6hfuw/default.jpg,False,False,Ini Talk Show adalah program talk show masa ki...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,_fcee58knGQ,Reza Arap Ngajak Ketemuan Ericko Lim! Jawaban ...,2020-03-25T05:29:02.000Z,UCSTxLbo0T-pFIZNjAPdkNtg,Youtuber Kepo,25,20.29.03,ericko lim|reza oktovian|ybrap|reza arap oktov...,135610,5542,217,603,https://i.ytimg.com/vi/_fcee58knGQ/default.jpg,False,False,Ericko Lim sudah bebas dan Reza Arap ngajak ke...
196,mv_nCtp-XIE,TRY NOT TO LAUGH : 23 Best Funny Moments Every...,2020-03-26T09:00:43.000Z,UCdhqVAZwe2NZ8fB9AtGPJZQ,T-FUN,26,20.29.03,Try not to laugh|fun|funny|funny moments|awkwa...,566130,3438,1071,27,https://i.ytimg.com/vi/mv_nCtp-XIE/default.jpg,False,False,TRY NOT TO LAUGH : 23 Best Funny Moments Every...
197,vxM4OuxUmTM,Top Lagu Pop Indonesia Terbaru 2020 Hits Pilih...,2020-03-24T22:00:11.000Z,UCC1Mg8YEIAd7bsoCCbxZq3A,nayan barari,22,20.29.03,lagu|lagu pop|lagu pop indonesia|lagu indonesi...,110364,681,102,28,https://i.ytimg.com/vi/vxM4OuxUmTM/default.jpg,False,False,Top Lagu Pop Indonesia Terbaru 2020 Hits Pilih...
198,trWp5v6-W0Y,SIA - CHANDELIER (COVER CHERYLL),2020-03-22T02:00:13.000Z,UCIqzqfsgUqctbM-MANntwuQ,Putih Abu-Abu,10,20.29.03,Putih Abu-abu|Cover Musik|Music Cover,1007069,49727,1699,6017,https://i.ytimg.com/vi/trWp5v6-W0Y/default.jpg,False,False,SIA - CHANDELIER (COVER CHERYLL)Original Song:...


In [58]:
df = pd.concat([df1, df2])
df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
0,o4MCAySZ0Hg,"LET'S MAKE IT HAPPEN, 2020",2020-03-27T06:02:30.000Z,UCepgnl-TtJ8DurHdC6EE22w,Ria SW,24,20.28.03,[none],1110686,126750,1149,10526,https://i.ytimg.com/vi/o4MCAySZ0Hg/default.jpg,False,False,Tahun 2019 adalah tahun paling berat buatku. A...
1,sVWyQUZOmLc,GOOGLE KALAH TEKA TEKI||BOCAH KORSLET,2020-03-23T13:47:16.000Z,UCMJozRNsVN9KkaUXokUP9eQ,Ronsi Geronsiyono,24,20.28.03,Bocah Korslet|Bocah|Korslet|Bahasa Inggris|kom...,8442248,257400,13168,12000,https://i.ytimg.com/vi/sVWyQUZOmLc/default.jpg,False,False,Terima kasih sudah menontonJangan lupa subscri...
2,ZdjSIBjDspU,TEGAS! Pesan Persatuan dan Kritik Dokter Tirta...,2020-03-24T23:00:11.000Z,UCYEa4_MC7OFjcWrO6SO-u8g,Indonesia Lawyers Club,25,20.28.03,tvone|tv one|tvonenews|indonesia lawyers club|...,3627883,131862,2676,20810,https://i.ytimg.com/vi/ZdjSIBjDspU/default.jpg,False,False,"Jakarta, tvOnenews.com - Dokter Tirta menyampa..."
3,bBxWId6hfuw,Alasan Tegar Septian Menikah Diumur 18 Tahun,2020-03-25T06:30:02.000Z,UCo1CFjot3CX97sn_ngJnGPg,Ini Talk Show,23,20.28.03,net|media|netmedia|netmediatama|indonesia|jaka...,6365021,98615,4462,12162,https://i.ytimg.com/vi/bBxWId6hfuw/default.jpg,False,False,Ini Talk Show adalah program talk show masa ki...
4,DmJZNdlC8IQ,ORANG TINGGI VS ORANG PENDEK || Situasi Memalu...,2020-03-24T07:00:04.000Z,UCGXnDxirLuSrIHwQXWxRzeg,123 GO! Like Indonesian,24,20.28.03,123 go|123 GO!|123 GO! Like|123 GO! Like Indon...,1625720,14068,2174,911,https://i.ytimg.com/vi/DmJZNdlC8IQ/default.jpg,False,False,"Orang tinggi punya masalah, orang pendek juga ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,_fcee58knGQ,Reza Arap Ngajak Ketemuan Ericko Lim! Jawaban ...,2020-03-25T05:29:02.000Z,UCSTxLbo0T-pFIZNjAPdkNtg,Youtuber Kepo,25,20.29.03,ericko lim|reza oktovian|ybrap|reza arap oktov...,135610,5542,217,603,https://i.ytimg.com/vi/_fcee58knGQ/default.jpg,False,False,Ericko Lim sudah bebas dan Reza Arap ngajak ke...
196,mv_nCtp-XIE,TRY NOT TO LAUGH : 23 Best Funny Moments Every...,2020-03-26T09:00:43.000Z,UCdhqVAZwe2NZ8fB9AtGPJZQ,T-FUN,26,20.29.03,Try not to laugh|fun|funny|funny moments|awkwa...,566130,3438,1071,27,https://i.ytimg.com/vi/mv_nCtp-XIE/default.jpg,False,False,TRY NOT TO LAUGH : 23 Best Funny Moments Every...
197,vxM4OuxUmTM,Top Lagu Pop Indonesia Terbaru 2020 Hits Pilih...,2020-03-24T22:00:11.000Z,UCC1Mg8YEIAd7bsoCCbxZq3A,nayan barari,22,20.29.03,lagu|lagu pop|lagu pop indonesia|lagu indonesi...,110364,681,102,28,https://i.ytimg.com/vi/vxM4OuxUmTM/default.jpg,False,False,Top Lagu Pop Indonesia Terbaru 2020 Hits Pilih...
198,trWp5v6-W0Y,SIA - CHANDELIER (COVER CHERYLL),2020-03-22T02:00:13.000Z,UCIqzqfsgUqctbM-MANntwuQ,Putih Abu-Abu,10,20.29.03,Putih Abu-abu|Cover Musik|Music Cover,1007069,49727,1699,6017,https://i.ytimg.com/vi/trWp5v6-W0Y/default.jpg,False,False,SIA - CHANDELIER (COVER CHERYLL)Original Song:...


In [59]:
# is there a missing value?
df.isna().sum()

video_id              0
title                 0
publishedAt           0
channelId             0
channelTitle          0
                     ..
comment_count         0
thumbnail_link        0
comments_disabled     0
ratings_disabled      0
description          13
Length: 16, dtype: int64

In [60]:
# Displaying column info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 0 to 199
Data columns (total 16 columns):
video_id             350 non-null object
title                350 non-null object
publishedAt          350 non-null object
channelId            350 non-null object
channelTitle         350 non-null object
categoryId           350 non-null int64
trending_date        350 non-null object
tags                 350 non-null object
view_count           350 non-null int64
likes                350 non-null int64
dislikes             350 non-null int64
comment_count        350 non-null int64
thumbnail_link       350 non-null object
comments_disabled    350 non-null bool
ratings_disabled     350 non-null bool
description          337 non-null object
dtypes: bool(2), int64(5), object(9)
memory usage: 41.7+ KB


# Feature Extraction

The dataset does not have labels for how long a video stays in the trending list. Therefore, the data is reformatted for the purpose of this project before preforming data extraction